In [9]:
import sys
from pathlib import Path

project_root = Path.cwd()
sys.path.append(str(project_root))
sys.path.append(str(project_root / 'src'))
sys.path.append(str(project_root / 'src' / 'data_processing'))
sys.path.append(str(project_root / 'src' / 'fine_tuning'))


Simplified ID Strategy\n
Source|Current|NewID|Example
QA  no id {video_id}_{order} 5R0FIkU97HY_01
Interview   has chunk_id    already unique  int_{interview_id}_17
Article     has chunk_id    already unique  art_{article_id}_04

In [ ]:
import json
from pathlib import Path
from collections import defaultdict

def process_qa_file(input_path: str, output_path: str):
    """Add order-based IDs to QA pairs"""
    
    items = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():  # Skip empty lines
                items.append(json.loads(line))
    
    # Count per video
    video_counter = defaultdict(int)
    
    for item in items:
        video_id = item['video_id']
        video_counter[video_id] += 1
        item['id'] = f"{video_id}_{video_counter[video_id]:02d}"
    
    with open(output_path, 'w', encoding='utf-8') as f:
        for item in items:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"✓ QA: {len(items)} items, {len(video_counter)} videos")




Run

In [1]:
import json

lines = open('./data/fine_tuning/train_data.jsonl', encoding='utf-8').readlines()
print(f"Total lines: {len(lines)}")

for i, line in enumerate(lines):
    if line.strip():
        r = json.loads(line)
        chars = len(r['input'] + r['output'])
        print(f"{i+1}: {chars} chars (~{chars//3} tokens)")

Total lines: 10
1: 1320 chars (~440 tokens)
2: 3632 chars (~1210 tokens)
3: 1786 chars (~595 tokens)
4: 2405 chars (~801 tokens)
5: 4532 chars (~1510 tokens)
6: 3583 chars (~1194 tokens)
7: 1235 chars (~411 tokens)
8: 4309 chars (~1436 tokens)
9: 2235 chars (~745 tokens)
10: 2359 chars (~786 tokens)


In [3]:
import json

# Check article format
with open('./data/processed/articles_with_questions.jsonl', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if line.strip() and i < 2:
            r = json.loads(line)
            print(f"=== Article {i+1} ===")
            print(f"Questions: {r.get('potential_questions', [])}")
            print(f"Text preview: {r['text'][:300]}...")
            print()

=== Article 1 ===
Questions: ['Что значит быть женственной и при этом оставаться сильной и независимой женщиной?', 'Как не потерять женственность, когда хочешь быть самодостаточной и уверенной в себе?', 'Почему образ успешной и самостоятельной женщины воспринимается как «мужик в юбке»?']
Text preview: "Быть женственной не значит быть тряпкой" Когда я говорю о самодостаточной и уверенной в себе женщине, многие возражают: ведь получится мужик в юбке! Давайте об этом поговорим. У многих женщин, читающих мои колонки и слушающих лекции, возникает протест. Когда я говорю о независимости, самодостаточно...

=== Article 2 ===
Questions: ['Почему мужчина начинает раздражаться из-за постоянных финансовых просьб женщины?', 'Как деньги влияют на власть и отношения в семье между мужчиной и женщиной?', 'Почему с возрастом у женщины снижается интерес мужчины, если у нее нет собственной жизни?']
Text preview: Женщина как постоянный источник расходов постепенно начинает раздражать. Ее нужды кажутся не 

prepare qa_pairs to train and test data for ft

In [2]:
import json
import random

# Load existing train_data (your 10 examples)
train_data = []
with open('./data/fine_tuning/train_data.jsonl', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            train_data.append(json.loads(line))

print(f"Existing train examples: {len(train_data)}")

# Load qa_pairs
qa_pairs = []
with open('./data/processed/qa_pairs.jsonl', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            r = json.loads(line)
            qa_pairs.append({
                "input": r['question'],
                "output": r['answer']
            })

print(f"QA pairs: {len(qa_pairs)}")

# Split qa_pairs: 90% train, 10% test
random.seed(42)
random.shuffle(qa_pairs)
split = int(len(qa_pairs) * 0.9)

# Combine
train_data.extend(qa_pairs[:split])
test_data = qa_pairs[split:]

print(f"Total train: {len(train_data)}")
print(f"Test: {len(test_data)}")

# Save
with open('./data/fine_tuning/train_data.jsonl', 'w', encoding='utf-8') as f:
    for item in train_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

with open('./data/fine_tuning/test_data.jsonl', 'w', encoding='utf-8') as f:
    for item in test_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print("✅ Saved!")

Existing train examples: 10
QA pairs: 241
Total train: 226
Test: 25
✅ Saved!


In [5]:
import json

# Add articles - only FIRST question per chunk
articles_added = 0
with open('./data/processed/articles_with_questions.jsonl', encoding='utf-8') as f:
    articles = [json.loads(line) for line in f if line.strip()]

with open('./data/fine_tuning/train_data.jsonl', 'a', encoding='utf-8') as f:
    for r in articles:
        questions = r.get('potential_questions', [])
        if questions:
            item = {
                "input": questions[0],  # Only first question
                "output": r['text']
            }
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
            articles_added += 1

print(f"Articles added: {articles_added}")

# Count total
with open('./data/fine_tuning/train_data.jsonl', encoding='utf-8') as f:
    total = len([l for l in f if l.strip()])
print(f"Total train examples: {total}")

Articles added: 250
Total train examples: 476


working on interview_ft.jsonl

In [7]:
import json

# Read entire file as one JSON or multiple JSON objects
with open('./data/processed/intervie_ft.jsonl', encoding='utf-8') as f:
    content = f.read()

# Try to parse - might be array or single objects separated by whitespace
interviews = []
decoder = json.JSONDecoder()
content = content.strip()
pos = 0

while pos < len(content):
    try:
        obj, end = decoder.raw_decode(content, pos)
        interviews.append(obj)
        pos = end
        # Skip whitespace
        while pos < len(content) and content[pos] in ' \n\t\r':
            pos += 1
    except json.JSONDecodeError:
        break

print(f"Loaded interviews: {len(interviews)}")

# Add to train_data
with open('./data/fine_tuning/train_data.jsonl', 'a', encoding='utf-8') as f:
    for r in interviews:
        messages = r.get('messages', [])
        user_msg = None
        assistant_msg = None
        for m in messages:
            if m['role'] == 'user':
                user_msg = m['content']
            elif m['role'] == 'assistant':
                assistant_msg = m['content']
        
        if user_msg and assistant_msg:
            item = {
                "input": user_msg,
                "output": assistant_msg
            }
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

# Count total
with open('./data/fine_tuning/train_data.jsonl', encoding='utf-8') as f:
    total = len([l for l in f if l.strip()])
print(f"Total train examples: {total}")
print("✅ Done!")

Loaded interviews: 18
Total train examples: 494
✅ Done!


In [1]:
# Check data balance
import json

qa_count = 0
article_count = 0
short_answers = 0
long_answers = 0

with open('./data/fine_tuning/train_data.jsonl', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            r = json.loads(line)
            out_len = len(r['output'])
            
            if out_len > 2000:
                article_count += 1
                long_answers += 1
            else:
                qa_count += 1
            
            if out_len < 500:
                short_answers += 1

print(f"Short answers (<500 chars): {short_answers}")
print(f"Long answers (>2000 chars): {long_answers}")
print(f"Likely Q&A: {qa_count}")
print(f"Likely articles: {article_count}")

Short answers (<500 chars): 127
Long answers (>2000 chars): 21
Likely Q&A: 473
Likely articles: 21


In [3]:
import json

# Append qa_pairs to existing 10
with open('./data/processed/qa_pairs.jsonl', encoding='utf-8') as f:
    qa_pairs = [json.loads(line) for line in f if line.strip()]

with open('./data/fine_tuning/train_data.jsonl', 'a', encoding='utf-8') as f:
    for r in qa_pairs:
        item = {"input": r['question'], "output": r['answer']}
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

# Count total
with open('./data/fine_tuning/train_data.jsonl', encoding='utf-8') as f:
    total = len([l for l in f if l.strip()])
    
print(f"Total: {total}")  # Should be 251

Total: 250


In [6]:
import json
from pathlib import Path
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
data_path = Path("data/fine_tuning/train_data.jsonl")
output_path = Path("data/fine_tuning/train_data_clean.jsonl")

MIN_ANSWER_TOKENS = 50  # Minimum for a "real" answer with substance

kept = []
removed = []

with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        r = json.loads(line)
        answer_tokens = len(tokenizer.encode(r['output']))
        
        if answer_tokens >= MIN_ANSWER_TOKENS:
            kept.append(r)
        else:
            removed.append({
                'tokens': answer_tokens,
                'q': r['input'][:60],
                'a': r['output'][:100]
            })

# Save cleaned data
with open(output_path, 'w', encoding='utf-8') as f:
    for r in kept:
        f.write(json.dumps(r, ensure_ascii=False) + '\n')

# Report
print(f"✅ Kept: {len(kept)} examples")
print(f"❌ Removed: {len(removed)} examples\n")

print("Removed examples:")
for ex in sorted(removed, key=lambda x: x['tokens']):
    print(f"  [{ex['tokens']} tok] Q: {ex['q']}...")
    print(f"           A: {ex['a']}")
    print()

✅ Kept: 245 examples
❌ Removed: 6 examples

Removed examples:
  [3 tok] Q: Как жить с мужем-невротиком и быть счастливой?...
           A: Никак.

  [9 tok] Q: Не хочется выходить из дома. Это социофобия?...
           A: Нет, это депрессия.

  [12 tok] Q: У меня есть муж и есть родители. Я зарабатываю достаточно мн...
           A: Может, но не должен, я так считаю.

  [28 tok] Q: Мне 20 лет. Я жила с бабушкой и дедушкой, с мамой и папой об...
           A: А в школе влюблялись в кого-нибудь? Если да — будете жить нормально. Это нормально.

  [37 tok] Q: Есть ли такое понятие, как здоровая и нездоровая разница в в...
           A: Нет проблем. Это дело каждого. Я вообще в женском возрасте не понимаю — мне что 60, что 25, одно и т

  [46 tok] Q: После смерти тёти, с которой я была очень близка, мне очень ...
           A: Ваша тётя была бы рада, если бы вы перестали горевать, потому что если бы она была жива, она бы очен



In [13]:
from inference_lora import load_model, ask

model, tokenizer = load_model()

prompt = """Generate 2-3 questions (in Russian) for this text:

Совет «делать только то, что хочется» наши граждане воспринимают как призыв к анархии. Свои самые сильные желания они считают непременно низменными, порочными, опасными для окружающих. Люди уверены, что они тайные беспредельщики, и попросту боятся дать себе волю! Я вижу в этом серьезный симптом всеобщего невроза.\n\nГоворишь человеку: «Делай то, что хочешь!» А он: «Ну что вы! Разве так можно?!»\n\nОтвечаю: «Если вы считаете себя хорошим человеком, то да. Можно и нужно». Желания хорошего человека совпадают с интересами окружающих.\n\nШесть правил, которые помогли не одному десятку людей выйти из невроза, – результат 30 лет практики. Это не значит, что я думал над ними 30 лет. Скорее, однажды они сами стихийно выстроились, как таблица Менделеева в голове Менделеева, когда он проснулся.\n\nПравила на первый взгляд простые:\n\n1. Делать только то, что хочется.\n\n2. Не делать того, чего делать не хочется.\n\n3. Сразу говорить о том, что не нравится.\n\n4. Не отвечать, когда не спрашивают.\n\n5. Отвечать только на вопрос.\n\n6. Выясняя отношения, говорить только о себе.

Response - JSON array ONLY:
["вопрос 1", "вопрос 2", "вопрос 3"]"""

response = ask(model, tokenizer, prompt)
print(response)

🤖 Loading model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.16s/it]


✅ Ready
[
    "Что такое невроз по вашему мнению?",
    "Какие из шести правил вам сложно реализовать и почему?",
    "Можете рассказать про шестое правило подробнее?"


In [3]:
import json
from pathlib import Path

DATA_DIR = Path(r"C:\Projects\projects_py\labkovsky-model\data\fine_tuning")

# Input files
QA_FILE = DATA_DIR / "train_data_clean.jsonl"
DIALOGUES_FILE = Path(r"C:\Projects\projects_py\labkovsky-model\src\fine_tuning\data\Hochu_i_budu_dialogues.jsonl")

# Output
OUTPUT_FILE = DATA_DIR / "train_data_final.jsonl"

all_records = []

# 1. Q&A → messages
with open(QA_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            r = json.loads(line)
            all_records.append({
                "messages": [
                    {"role": "user", "content": r["input"]},
                    {"role": "assistant", "content": r["output"]}
                ]
            })

print(f"Q&A: {len(all_records)}")

# 2. Dialogues → messages (rename turns to messages)
count = 0
with open(DIALOGUES_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            r = json.loads(line)
            all_records.append({
                "messages": r["turns"]
            })
            count += 1

print(f"Dialogues: {count}")
print(f"Total: {len(all_records)}")

# Save
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    for r in all_records:
        f.write(json.dumps(r, ensure_ascii=False) + '\n')

print(f"Saved to {OUTPUT_FILE}")

Q&A: 245
Dialogues: 44
Total: 289
Saved to C:\Projects\projects_py\labkovsky-model\data\fine_tuning\train_data_final.jsonl


In [ ]:
import json
from collections import Counter, defaultdict

def collect_decision_types(jsonl_path: str):
    decision_types = []
    tags_by_dt = defaultdict(list)  # dt -> list of all tags
    
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError as e:
                print(f"[WARN] Line {line_num}: invalid JSON ({e})")
                continue
            
            dt = obj.get("decision_type")
            aux_tags = obj.get("aux_tags", [])
            
            if dt is not None:
                decision_types.append(dt)
                tags_by_dt[dt].extend(aux_tags)
    
    # Count decision types
    dt_counter = Counter(decision_types)
    
    print("\n=== DECISION TYPES ===")
    for dt, count in dt_counter.most_common():
        print(f"{dt}: {count}")

    
    return dt_counter, tags_by_dt

if __name__ == "__main__":
    collect_decision_types(r".\data\fine_tuning\qa_dt_corpus.jsonl")


=== DECISION TYPES ===
SELF_ESTEEM_CORRECTIVE: 81
EXPLANATION: 65
DEPENDENCY_BOUNDARIES: 42
ANXIETY_MANAGEMENT: 20
ADDICTION_PATTERN: 14
CLINICAL_ESCALATION: 13
AFFECTIVE_ADDICTION: 12
PARENTING_MODEL: 12
FEAR_SCENARIO_COPING: 11
PARENTING_LIMITS: 11


In [7]:
import json
def count_ft(jsonl_path: str):
    # count how many have short_answer (FT) and only RAG (tier)  and how many all cases we have and how many we have that good for FT that is all - short - rag
    countShort = 0
    countOnlyRAG = 0
    countALL = 0
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError as e:
                print(f"[WARN] Line {line_num}: invalid JSON ({e})")
                continue
            
            rag_only = obj.get("tier")
            short_answer = obj.get("short_answer")
                        
            if rag_only is not None:
                countOnlyRAG += 1
            if short_answer is not None:
                countShort += 1            
            countALL += 1

    print("\n=== TIER COUNTS ===")
    print(f"Only RAG: {countOnlyRAG}")
    print(f"Short Answer: {countShort}")
    print(f"ALL: {countALL}")
    print(f"Good for FT: {countALL - countShort - countOnlyRAG}")

    
    return countShort, countOnlyRAG

if __name__ == "__main__":
    count_ft(r".\data\fine_tuning\qa_rs_corpus_short.jsonl")


=== TIER COUNTS ===
Only RAG: 12
Short Answer: 7
ALL: 467
Good for FT: 448


In [6]:
import json

with open('C:/Projects/projects_py/labkovsky-model/data/fine_tuning/qa_corpus_rag_optimized.jsonl', encoding='utf-8') as f:
    for i, line in enumerate(f, 1):
        line = line.strip()
        if not line:
            continue
        try:
            item = json.loads(line)
            if item.get('chunk_role') == 'INTERVENTION':
                words = len(item['answer'].split())
                if words > 100:
                    print(f"{item['id']}: {words} слов — проверить")
        except json.JSONDecodeError as e:
            print(f"Ошибка в строке {i}: {e}")

__yreUuBY_g_03_interv: 113 слов — проверить
WVFDJxAyqs8_03_interv: 107 слов — проверить
WVFDJxAyqs8_06_interv: 105 слов — проверить
6wK5Roh_tvY_02_interv: 108 слов — проверить
0LHoKduq8XI_02_interv: 109 слов — проверить
0LHoKduq8XI_03_interv: 104 слов — проверить
-aCsNAR0w38_01_interv: 106 слов — проверить
aLQt7RU63gk_05_interv: 116 слов — проверить
W5dINgfexk8_05_interv: 124 слов — проверить
XrtG6NyMeU4_10_interv: 108 слов — проверить
XrtG6NyMeU4_15_interv: 118 слов — проверить
QR-gHOvk3WM_05_interv: 124 слов — проверить
QR-gHOvk3WM_07_interv: 111 слов — проверить
QR-gHOvk3WM_08_interv: 117 слов — проверить
QR-gHOvk3WM_09_interv: 163 слов — проверить
QR-gHOvk3WM_10_interv: 130 слов — проверить
QR-gHOvk3WM_13_interv: 114 слов — проверить
QR-gHOvk3WM_14_interv: 130 слов — проверить
QR-gHOvk3WM_15_interv: 110 слов — проверить
fKBFvwFjUks_02_interv: 129 слов — проверить
fKBFvwFjUks_04_interv: 157 слов — проверить
fKBFvwFjUks_08_interv: 131 слов — проверить
fKBFvwFjUks_10_interv: 122 слов 

In [1]:
import chromadb

client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_collection("labkovsky")

# Check a sample
sample = collection.get(limit=3, include=["metadatas"])
for meta in sample["metadatas"]:
    print(meta.get("video_id"), meta.get("chunk_role"))

srJvn19GKNA EXPLANATION
srJvn19GKNA ESCALATION
srJvn19GKNA INTERVENTION


In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-YandexGPT-5-Lite-8B-it")

messages = [
    {"role": "system", "content": "Test"},
    {"role": "documents", "content": '[{"doc_id": 0, "content": "test"}]'},
    {"role": "user", "content": "Question"},
]

print(tokenizer.apply_chat_template(messages, tokenize=False))

c:\Projects\projects_py\labkovsky-model\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Projects\projects_py\labkovsky-model\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--Vikhrmodels--Vikhr-YandexGPT-5-Lite-8B-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admi

<s>system
Test</s>
<s>documents
[{"doc_id": 0, "content": "test"}]</s>
<s>user
Question</s>



In [2]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Load model with 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_name = "Vikhrmodels/Vikhr-YandexGPT-5-Lite-8B-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# Your RAG documents
documents = [
    {"doc_id": 0, "title": "EXPLANATION", "content": "В основе всех ваших зависимостей лежит тревога. Телефон, траты, алкоголь и зависимость от человека — это разные способы забить одно и то же состояние."},
    {"doc_id": 1, "title": "INTERVENTION", "content": "Признайте, что не можете справиться сами. Скажите себе: я не контролирую это. Потом решите — к специалисту или продолжать так жить."},
]

messages = [
    {"role": "system", "content": "Ты — Михаил Лабковский. Отвечай кратко и прямо, используя информацию из документов."},
    {"role": "documents", "content": json.dumps(documents, ensure_ascii=False)},
    {"role": "user", "content": "У меня 4 зависимости: траты, алкоголь, женатый мужчина, телефон. С чего начать?"},
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(model.device)

outputs = model.generate(
    inputs,
    max_new_tokens=256,
    temperature=0.3,
    do_sample=True,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Loading checkpoin

system
Ты — Михаил Лабковский. Отвечай кратко и прямо, используя информацию из документов.
documents
[{"doc_id": 0, "title": "EXPLANATION", "content": "В основе всех ваших зависимостей лежит тревога. Телефон, траты, алкоголь и зависимость от человека — это разные способы забить одно и то же состояние."}, {"doc_id": 1, "title": "INTERVENTION", "content": "Признайте, что не можете справиться сами. Скажите себе: я не контролирую это. Потом решите — к специалисту или продолжать так жить."}]
user
У меня 4 зависимости: траты, алкоголь, женатый мужчина, телефон. С чего начать?
assistant
Начать стоит с признания, что вы не можете справиться с этими зависимостями самостоятельно. Признайте, что вы не контролируете ситуацию. Затем примите решение: либо обратиться за помощью к специалисту, либо продолжать жить так, как есть.
